# Interpolative Seperable Density Fitting (ISDF) 

The ISDF implementation currently provides a THC-like factorization of the two electron integrals which should converge to the FFTDF representation of the ERIs in the limit of large THC rank. This differs from the assumption of using RSGDF throughout the rest of the resource estimation scripts. However, we typically are only interested in ISDF as an initial guess for the THC factors which are then subsequently reoptimized to regularize $\lambda$. The assumption here is that FFTDF / ISDF is a good enough approximation to the RSGDF ERIs and thus serves as a good initial guess.

Let's start by comparing the ISDF-MP2 energy as a function of the THC rank parameter. Recall that $M = c_\mathrm{THC} N/2$, where $c_\mathrm{THC}$ is the THC rank parameter and $N$ is the number of spin orbitals. $M$ is what we call num_thc in the code. 

It's important to recall what we are doing in the ISDF algorithm, that is we solve

\begin{equation}

u_{p\mathrm{k}}^*(\mathbf{r}_i) u_{q\mathbf{k}'}(\mathbf{r}_i) = \sum_\mu^M \xi_\mu(\mathbf{r}_i) u_{p\mathrm{k}}^*(\mathbf{r}_\mu) u_{q\mathbf{k}'}(\mathbf{r}_\mu)

\end{equation}

for $\xi_\mu(\mathbf{r}_i)$ given a set of interpolating points $(\{r_\mu\})$ which are selected from the original real space $(\{\mathbf{r}_i\})$ (FFT) grid of size $N_g$ using the KMeans-CVT algorithm.

For the purposes of this notebook it is helpful to use a value of $N_g$ which is smaller than that required to fully converge the FFTDF error. We will investigate this more at the end of the tutorial. 

In [ ]:
import os
import numpy as np
from ase.build import bulk

from pyscf.pbc import gto, scf
from pyscf.pbc.tools import pyscf_ase
from pyscf.pbc.mp import KMP2 


ase_atom = bulk("C", "diamond", a=3.5)
cell = gto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(ase_atom)
cell.a = ase_atom.cell[:].copy()
cell.basis = "gth-szv"
cell.pseudo = "gth-hf-rev"
# Use a smaller value of Ng that would otherwise be suggested (~ 26^3) for
# expediency + to allow exact ISDF factorization for comparison.
cell.mesh = [11]*3
cell.verbose = 0
cell.build()

kmesh = [1, 1, 3]
kpts = cell.make_kpts(kmesh)
num_kpts = len(kpts)
mf = scf.KRHF(cell, kpts)
mf.kernel()
print("SCF energy: ", mf.e_tot)

# converged SCF energy with appropriate Ng = -10.388904514046914, mesh = 28^3


Now let's find the ISDF THC factors using the KMeans-CVT algorithm to find the interpolating points. It's easiest to use the helper function `solve_kmeans_kpisdf` which will perform the necessary steps. 

In [ ]:
import openfermion.resource_estimates.pbc.thc as kthc
# Let's use the whole real space grid for some correctness checks first.
num_thc = np.prod(cell.mesh)
kpt_thc = kthc.solve_kmeans_kpisdf(mf, num_thc, verbose=False)
print(kpt_thc.__dict__.keys())

We see that the kpt_thc class has 4 attributes, `chi`, `xi`, `zeta` and `G_mapping`. `chi` corresponds to the cell periodic part of the Bloch orbital (i.e. $u_{p\mathbf{k}}(\mathbf{r}_\mu))$. `xi` corresponds to $\xi_{\mu}(\mathbf{r})$ in Eq. (1) above. To understand `zeta` and `G_mapping` it is helpful to recall we want to build

$$

(p\mathbf{k}pq\mathbf{k}-\mathbf{Q}|r\mathbf{k}'-\mathbf{Q} s\mathbf{k}') = \sum_{\mu\nu} u^*_{p\mathbf{k}}(\mathbf{r}_\mu))u_{p\mathbf{k}-\mathbf{{Q}}}(\mathbf{r}_\mu) \zeta_{\mu\nu}^{\mathbf{Q}\Delta \mathbf{G}_{\mathbf{Q}\mathbf{k}-\mathbf{Q}}\Delta\mathbf{G}_{\mathbf{Q}\mathbf{k}'-\mathbf{Q}}} u^*_{p\mathbf{k}'}(\mathbf{r}_\nu)u_{p\mathbf{k}-\mathbf{Q}}(\mathbf{r}_\nu)

$$

So `zeta` corresponds to $\zeta_{\mu\nu}^{\mathbf{Q}\Delta \mathbf{G}_{\mathbf{Q}\mathbf{k}-\mathbf{Q}}\Delta\mathbf{G}_{\mathbf{Q}\mathbf{k}'-\mathbf{Q}}}$ above, and `G_mapping` is a 2D array yielding the appropriate $\Delta G$ index given an index for $\mathbf{Q}$ and $\mathbf{k}$.

Let's look at an example to see that everything is correct.

In [ ]:
from openfermion.resource_estimates.pbc.utils import build_momentum_transfer_mapping 

momentum_map = build_momentum_transfer_mapping(cell, kpts)
num_spatial_orbs = mf.mo_coeff[0].shape[-1]

# Pick a particular momentum transfer
Q_indx = 1
k_indx = 1
k_prime_indx = 0
k_minus_Q_indx = momentum_map[Q_indx, k_indx]
k_prime_minus_Q_indx = momentum_map[Q_indx, k_prime_indx]

eri_kindices = [k_indx, k_minus_Q_indx, k_prime_minus_Q_indx, k_prime_indx]
eri_mos = [mf.mo_coeff[kindx] for kindx in eri_kindices]
eri_kpts = [mf.kpts[kindx] for kindx in eri_kindices]

eri_exact = mf.with_df.ao2mo(eri_mos, eri_kpts, compact=False).reshape((num_spatial_orbs,)*4)


kthc_eri_helper = kthc.KPTHCHelperDoubleTranslation(chi=kpt_thc.chi, zeta=kpt_thc.zeta, kmf=mf)

eri_thc = kthc_eri_helper.get_eri(eri_kindices)
# Can also do
# eri_exact = kthc_eri_helper.get_eri_exact(eri_kindices)

assert np.allclose(eri_thc, eri_exact)


Now let's check convergence of the integral error, and corresponding MP2 error, with the THC dimension or equivalently the THC rank parameter.

In [ ]:
from openfermion.resource_estimates.pbc.utils import compute_emp2_approx, build_cc_inst
cc_inst = build_cc_inst(mf)
eri_exact = cc_inst.ao2mo()
emp2_exact, _, _ = cc_inst.init_amps(eri_exact)
emp2_exact += mf.e_tot
delta_eri = []
delta_mp2 = []
thc_ranks = np.arange(2, 20, 2)
for cthc in thc_ranks:
    num_thc = cthc * num_spatial_orbs
    kpt_thc = kthc.solve_kmeans_kpisdf(mf, num_thc, verbose=False)
    kthc_eri_helper = kthc.KPTHCHelperDoubleTranslation(chi=kpt_thc.chi, zeta=kpt_thc.zeta, kmf=mf)
    eri_thc = kthc_eri_helper.get_eri(eri_kindices)
    eri_exact = kthc_eri_helper.get_eri_exact(eri_kindices)
    # Note pyscf omits a normalization factor of 1/Nk in their definition of ERIs
    delta_eri.append(np.max(np.abs(eri_thc-eri_exact))/num_kpts)
    emp2_approx = compute_emp2_approx(mf, kthc_eri_helper)
    delta_mp2.append(emp2_exact - emp2_approx)


Let's look at the convergence of the integral error

In [ ]:
import matplotlib.pyplot as plt

plt.plot(thc_ranks, delta_eri, marker="o")
plt.yscale("log")
plt.xlabel(r"$c_{\mathrm{THC}}$")
plt.ylabel(r"max$|\Delta(pq|rs)|$")

Let's see how this corresponds to the MP2 error.

In [ ]:
plt.cla()
plt.plot(thc_ranks, np.abs(delta_mp2), marker="o")
plt.yscale("log")
plt.xlabel(r"$c_{\mathrm{THC}}$")
plt.ylabel(r"$|\Delta E_{\mathrm{MP2}}|$ (Ha)")

We see that apart from some non-monotonic behaviour (which is expected due to the non-linear nature of the ISDF procedure), that a relatively large rank parameter is required to obtain say $< 0.1$ mHa error per cell. Note this could likely be reduced by carefully selecting the orbital sets we perform ISDF on as oo, ov, and vv blocks exhibit different low-rank behaviour, but for quantum algorithms this is not relevant. 

### Optional: Effect of Mesh Density

You might be worried that we're cheating by only including 11^3 grid points, and that we're not saving much with the ranks we're choosing.

Let us first see the fraction of points these ranks correspond to.

In [ ]:
plt.plot(thc_ranks, 100*np.array(thc_ranks)*num_spatial_orbs/(11**3), marker="o")
plt.xlabel(r"$c_{\mathrm{THC}}$")
plt.ylabel("Percentage of real space points selected")

Now let us crank up the FFTDF accuracy and see if the results change significantly. This cell will take around 10 minutes to run.

In [ ]:

results = {11: [], 15: [], 19: [], 21: [], 28: []}
#results = {11: [], 15: []}
for mesh in list(results.keys()):
    ase_atom = bulk("C", "diamond", a=3.5)
    cell = gto.Cell()
    cell.atom = pyscf_ase.ase_atoms_to_pyscf(ase_atom)
    cell.a = ase_atom.cell[:].copy()
    cell.basis = "gth-szv"
    cell.pseudo = "gth-hf-rev"
    # Use a smaller value of Ng that would otherwise be suggested (~ 26^3) for
    # expediency + to allow exact ISDF factorization for comparison.
    cell.mesh = [mesh]*3
    cell.verbose = 0
    cell.build()

    kmesh = [1, 1, 3]
    kpts = cell.make_kpts(kmesh)
    num_kpts = len(kpts)
    mf = scf.KRHF(cell, kpts)
    mf.kernel()
    print("SCF energy: ", mf.e_tot)

    from pyscf.pbc.mp import KMP2 
    emp2_exact = KMP2(mf).kernel()[0] + mf.e_tot
    print("Ng = {}^3, MP2 Correlation energy: {}".format(mesh, emp2_exact-mf.e_tot))
    thc_ranks = np.arange(2, 20, 2)
    for cthc in thc_ranks:
        print(f"Running mesh = {mesh}, cthc = {cthc}")
        num_thc = cthc * num_spatial_orbs
        kpt_thc = kthc.solve_kmeans_kpisdf(mf, num_thc, verbose=False)
        kthc_eri_helper = kthc.KPTHCHelperDoubleTranslation(chi=kpt_thc.chi, zeta=kpt_thc.zeta, kmf=mf)
        emp2_approx = compute_emp2_approx(mf, kthc_eri_helper)
        # Note pyscf omits a normalization factor of 1/Nk in their definition of ERIs
        results[mesh].append(emp2_exact - emp2_approx)

    plt.plot(thc_ranks, np.abs(results[mesh]), marker="o", label=f"$N_g = {mesh}^3$")
plt.yscale("log")
plt.xlabel(r"$c_{\mathrm{THC}}$")
plt.legend()
plt.ylabel(r"$|\Delta E_{\mathrm{MP2}}|$ (Ha)")



### Optional: Effect of Basis Set 

Another concern is that the basis set size is tiny so maybe things get worse as the basis set increases. Let's look into it by increasing the basis set size but still use a fairly coarse FFT grid, as we've seen it's not super important. This will also take several minutes to run.


In [ ]:

basis_results = {"gth-szv": [], "gth-dzvp": [], "gth-tzvp": []}
for basis in list(basis_results.keys()):
    ase_atom = bulk("C", "diamond", a=3.5)
    cell = gto.Cell()
    cell.atom = pyscf_ase.ase_atoms_to_pyscf(ase_atom)
    cell.a = ase_atom.cell[:].copy()
    cell.basis = basis 
    cell.exp_to_discard = 0.1
    cell.pseudo = "gth-hf-rev"
    # Use a smaller value of Ng that would otherwise be suggested (~ 26^3) for
    # expediency + to allow exact ISDF factorization for comparison.
    cell.mesh = [15]*3
    cell.verbose = 0
    cell.build()

    kmesh = [1, 1, 3]
    kpts = cell.make_kpts(kmesh)
    num_kpts = len(kpts)
    mf = scf.KRHF(cell, kpts)
    mf.kernel()
    print("SCF energy: ", mf.e_tot)

    num_spatial_orbs = mf.mo_coeff[0].shape[-1]
    from pyscf.pbc.mp import KMP2 
    emp2_exact = KMP2(mf).kernel()[0] + mf.e_tot
    print("basis = {}, MP2 Correlation energy: {}".format(basis, emp2_exact-mf.e_tot))
    thc_ranks = np.arange(2, 20, 2)
    for cthc in thc_ranks:
        print(f"Running basis = {basis}, cthc = {cthc}")
        num_thc = cthc * num_spatial_orbs
        kpt_thc = kthc.solve_kmeans_kpisdf(mf, num_thc, verbose=False)
        kthc_eri_helper = kthc.KPTHCHelperDoubleTranslation(chi=kpt_thc.chi, zeta=kpt_thc.zeta, kmf=mf)
        emp2_approx = compute_emp2_approx(mf, kthc_eri_helper)
        # Note pyscf omits a normalization factor of 1/Nk in their definition of ERIs
        basis_results[basis].append(emp2_exact - emp2_approx)

    plt.plot(thc_ranks*num_spatial_orbs, np.abs(basis_results[basis]), marker="o", label=f"basis = {basis}")
plt.yscale("log")
plt.xlabel(r"$M$")
plt.legend()
plt.ylabel(r"$|\Delta E_{\mathrm{MP2}}|$ (Ha)")

In [ ]:

for basis in list(basis_results.keys()):
    plt.plot(thc_ranks, np.abs(basis_results[basis]), marker="o", label=f"basis = {basis}")
plt.yscale("log")
plt.xlabel(r"$c_{\mathrm{THC}}$")
plt.legend()
plt.ylabel(r"$|\Delta E_{\mathrm{MP2}}|$ (Ha)")

## Effect on $\lambda$ 

Now let us investigate the $\lambda$ dependence of our ISDF-THC factorization. We will revert back to the minimal example from earlier.

In [ ]:

ase_atom = bulk("C", "diamond", a=3.5)
cell = gto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(ase_atom)
cell.a = ase_atom.cell[:].copy()
cell.basis = "gth-dzv"
cell.pseudo = "gth-hf-rev"
# Use a smaller value of Ng that would otherwise be suggested (~ 26^3) for
# expediency + to allow exact ISDF factorization for comparison.
cell.mesh = [15]*3
cell.verbose = 0
cell.build()

kmesh = [1, 1, 3]
kpts = cell.make_kpts(kmesh)
num_kpts = len(kpts)
mf = scf.KRHF(cell, kpts)
mf.kernel()
print("SCF energy: ", mf.e_tot)

# converged SCF energy with appropriate Ng = -10.388904514046914, mesh = 28^3

hcore = np.asarray([C.conj().T @ hc @ C for C, hc in zip(mf.mo_coeff, mf.get_hcore())])
num_spatial_orbs = hcore.shape[-1]
thc_ranks = np.arange(2, 20, 2)
cc_inst = build_cc_inst(mf)
eri_exact = cc_inst.ao2mo()
emp2_exact, _, _ = cc_inst.init_amps(eri_exact)
np.random.seed(7)
print(f"FFTDF MP2 Correlation energy: {emp2_exact}")
for cthc in thc_ranks:
    num_thc = cthc * num_spatial_orbs
    kpt_thc = kthc.solve_kmeans_kpisdf(mf, num_thc, verbose=False)
    kthc_eri_helper = kthc.KPTHCHelperDoubleTranslation(chi=kpt_thc.chi, zeta=kpt_thc.zeta, kmf=mf)
    # Note pyscf omits a normalization factor of 1/Nk in their definition of ERIs
    thc_lambda = kthc.compute_lambda(hcore, kthc_eri_helper)
    emp2_approx = compute_emp2_approx(mf, kthc_eri_helper) - mf.e_tot
    emp2_error = abs(emp2_approx - emp2_exact)
    print(f"cthc = {cthc}, MP2 error: {emp2_error:4.3e}, lambda = {thc_lambda.lambda_total:4.3e}")
    if emp2_error < 1e-4:
        print(f"--> MP2 error < 0.1: {emp2_error:4.3e}, lambda = {thc_lambda.lambda_total:4.3e}")

Now we can try to improve $\lambda$ by reoptimizing the THC factors using our ISDF factors as an initial guess. Practically this should mean we can use a smaller THC rank parameter for comparable MP2 accuracy. Note reoptimizing the THC factors is quite expensive. This cell may take 20 minutes to run. You should see that for a $c_\mathrm{THC}=6$ the MP2 error is reduced by an order of magnitude. The optimization can be sped up by running on a GPU.

In [ ]:

from openfermion.resource_estimates.pbc import utils
# Recall we need RSGDF integrals to fit to.
rsmf = scf.KRHF(mf.cell, mf.kpts).rs_density_fit()
rsmf.kernel()
mymp = KMP2(rsmf)
emp2_rsgdf = mymp.kernel()[0]
hcore, Luv = utils.build_hamiltonian(rsmf)
np.random.seed(7)
print(f"RSGDF emp2: {emp2_rsgdf}")
cthc = 6
num_thc = cthc * num_spatial_orbs
# Here we use the helper function kpoint_thc_via_isdf which will first find
# the ISDF factors and feed these into the BFGS and AdaGrad solvers.
kpt_thc, _ = kthc.kpoint_thc_via_isdf(mf, Luv, num_thc, verbose=False)
kthc_eri_helper = kthc.KPTHCHelperDoubleTranslation(chi=kpt_thc.chi, zeta=kpt_thc.zeta, kmf=mf)
# Note pyscf omits a normalization factor of 1/Nk in their definition of ERIs
thc_lambda = kthc.compute_lambda(hcore, kthc_eri_helper)
emp2_approx = utils.compute_emp2_approx(mf, kthc_eri_helper) - mf.e_tot # only compare correlation energy.
emp2_error = abs(emp2_approx - emp2_rsgdf)
print(f"cthc = {cthc}, MP2 error: {emp2_error:4.3e}, lambda = {thc_lambda.lambda_total:4.3e}")